# Creating a P2WSH multisig transaction

In this section we'll create a P2WSH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - TestShell setup - '[P2PKH chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb)'

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/dariuscognac/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/dariuscognac/Documents/Github/bitcoin-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

### Create a P2WSH multisig UTXO

In order to create a transaction spending from a P2WSH multisig UTXO, we'll first need to create the UTXO that is locked with a multisig script. For this example we'll use a 2 of 2 multisig script. Note that this type of output will be identical to funding transactions used for layer 2 lightning channels. 

We'll create a two pubkeys and manually conscruct the multisig script. Then we'll fund it using the bitcoind wallet created in the setup step.

#### Create a multisig redeemScript 
For more on this step, review the 'Bitcoin Script' notebook.

In [2]:
privkey1 = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
pubkey1 = privkey_to_pubkey(privkey1)

privkey2 = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
pubkey2 = privkey_to_pubkey(privkey2)

# "21" is the length of a 33 byte (compressed) pubkey in hex notation
redeemScript = bytes.fromhex(
    "52" 
    + "21"
    + pubkey1.hex()
    + "21"
    + pubkey2.hex()
    + "52"
    + "ae")

print(redeemScript.hex())

5221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2752ae


#### Convert the redeemScript to a P2WSH address

For more on this step, review the 'Addresses' notebook.

In [3]:
address_to_spend = script_to_p2wsh(redeemScript, "regtest")
print(address_to_spend)

bcrt1qpqn5k3h89nfv6cnrkvk3rt3g0zfhqfz23cxkgapsenj29ety5ckqyrn25s


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

Now that we have the address, we'll fund it using the `TestShell` commands. For more on these steps you can view the [function definitions](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/functions/setup_testshell.py) or look at the first [P2PKH notebook](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb) example.

In [4]:
node = setup_testshell()
txid_to_spend, index_to_spend = fund_address(node, address_to_spend, 2.001)
print(f"txid: {txid_to_spend}, {index_to_spend}")

2022-12-29T01:40:28.907000Z TestFramework (INFO): Initializing test directory /var/folders/r5/yk8yg2xs1gs8xzkn5l8vr72w0000gn/T/bitcoin_func_test_amyyxq7f
txid: 1362ff4ca1cfefb688f3c8b2e965cfed68d8c1549d161162684920dd218b5498, 0


## Spending a P2WSH (multisig) UTXO

Now that we have some funds locked up in a P2WSH multisig utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE`.

From our previous examples we know this corresponds to a scriptPubkey of `76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac`, so we will skip the decoding step. For more on decoding base58 addresses, refer to the 'Addresses' chapter.

In [5]:
receiver_spk = bytes.fromhex("76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac")

### Create an unsigned P2WSH transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [6]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `privkey1`, `pubkey1`, ...

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("4444444444444444444444444444444444444444444444444444444444444444")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("76a914") + hash160(change_pubkey) + bytes.fromhex("88ac")

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [7]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + pushbytes(output1_spk)
    + output2_value
    + pushbytes(output2_spk)
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  020000000198548b21dd2049686211169d54c1d868edcf65e9b2c8f388b6efcfa14cff62130000000000ffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914cc1b07838e387deacd0e5232e1e8b49f4c29e48488ac00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (marker, flag, witness).

In [8]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "c243d2ca9343f1bbe24b8a92e93ad37758f65428cce9a8f9b8ffb1913ac691f4",
  "hash": "c243d2ca9343f1bbe24b8a92e93ad37758f65428cce9a8f9b8ffb1913ac691f4",
  "version": 2,
  "size": 119,
  "vsize": 119,
  "weight": 476,
  "locktime": 0,
  "vin": [
    {
      "txid": "1362ff4ca1cfefb688f3c8b2e965cfed68d8c1549d161162684920dd218b5498",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "1.50000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 3bc28d6d92d9073fb5e3adf481795eaf446bceed OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE)#xlnzfr97",
        "hex": "76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac",
        "address": "mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE",
        "type": "pubkeyhash"
      }
    },
    {
      "value": "0.50000000",
      "n": 1,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 cc1

Segwit transactions have a new signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)

In [9]:
scriptcode = redeemScript

input_amount_sat = int(2.001 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

02000000bfb56f6bdd6a0a677fe1f6e58a3e8ce287e6618d2bf487adf7b2b5fb3f6049113bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e7066504498548b21dd2049686211169d54c1d868edcf65e9b2c8f388b6efcfa14cff621300000000475221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2752aea048ed0b00000000ffffffff7c27b92041484c01942ac85d0fe5ff31e92d1a54e1618e190ef3f316c4799ecf0000000001000000


Now we are ready to hash this transaction and produce two ecdsa signatures on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [10]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input privkey1
signing_key1 = ecdsa.SigningKey.from_string(privkey1, curve=ecdsa.SECP256k1) 
signature1 = signing_key1.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature1 = signature1 + bytes.fromhex("01")

# Sign the sigHash with the input privkey2
signing_key2 = ecdsa.SigningKey.from_string(privkey2, curve=ecdsa.SECP256k1) 
signature2 = signing_key2.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature2 = signature2 + bytes.fromhex("01")

# Combine the signatures into the final scriptSig
witness = (
    # indicate the number of stack items
    bytes.fromhex("04")
    + bytes.fromhex("00") # Add an extra "00" for the CheckMultiSig bug
    + varint_len(signature1)
    + signature1
    + varint_len(signature2)
    + signature2
    + varint_len(redeemScript)
    + redeemScript
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  0200000000010198548b21dd2049686211169d54c1d868edcf65e9b2c8f388b6efcfa14cff62130000000000ffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914cc1b07838e387deacd0e5232e1e8b49f4c29e48488ac040047304402207894d3ba08df43e2a35e6eadbad445ad62a595a82b746a30185c9b46d0ec4b370220312601e8d6fad65735b2e0b625aa6bfbebbe15e6884021222f7c46429cf2519a0147304402207183b76dab31b3173c9822fd62f7efb1faf1c45c3d8237e13690eba04cfea3f502204657fb32d9ff731afad2c9b55a99bbc54fa49c6de1676f41122cb7b5131e9d0301475221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2752ae00000000


### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [11]:
new_tx_txid = node.sendrawtransaction(signed_tx.hex())
# result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(new_tx_txid)

c243d2ca9343f1bbe24b8a92e93ad37758f65428cce9a8f9b8ffb1913ac691f4


In [12]:
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "c243d2ca9343f1bbe24b8a92e93ad37758f65428cce9a8f9b8ffb1913ac691f4",
  "hash": "eb53fe0f9c4b5edec8f81d5b05da2b49af2d4306f81328746d4c53d16762aaec",
  "version": 2,
  "size": 339,
  "vsize": 174,
  "weight": 696,
  "locktime": 0,
  "vin": [
    {
      "txid": "1362ff4ca1cfefb688f3c8b2e965cfed68d8c1549d161162684920dd218b5498",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "",
        "304402207894d3ba08df43e2a35e6eadbad445ad62a595a82b746a30185c9b46d0ec4b370220312601e8d6fad65735b2e0b625aa6bfbebbe15e6884021222f7c46429cf2519a01",
        "304402207183b76dab31b3173c9822fd62f7efb1faf1c45c3d8237e13690eba04cfea3f502204657fb32d9ff731afad2c9b55a99bbc54fa49c6de1676f41122cb7b5131e9d0301",
        "5221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2752ae"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 

## Quiz

1. How does the scriptPubKey for a P2WSH output compare to a P2SH output?
2. If we generated a new valid signature for `signature2` and replaced it in `signed_tx`, would the resulting transaction have the same or a different transaction ID? Why is this property important for lightning payment channels?

## Answers

1. P2SH scriptPubKeys use a 20-byte HASH160 digest, whereas P2WSH uses a 32-byte HASH256 digest. For an explanation of the security benefits of P2WSH outputs see this [Stack Exchange Answer](https://bitcoin.stackexchange.com/questions/106140/why-are-p2wsh-addresses-larger-than-p2sh-addresses).
2. The resulting transaction ID would be the same. This property (non-transaction malleability) is important for lightning payment channels as it allows parties to create offline transaction dependency chains.

## Exercise



Inspired by this [Stack Exchange Post](https://bitcoin.stackexchange.com/questions/115443/1-of-1-multisig), try creating a 1 of 1 multisig address using `pubkey1`. The address should be:
`bcrt1q6vaj75sc5ygw4hcpjwrw4e52k7fglsdwdr2jpyyw5cx9d64nxfgs9es26d`

Try sending some bitcoin to that address and spending from it. The code used should be the same as the example above, but with some small modifications to the witness field. 